In [5]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
from pathlib import Path
import os
import re
import numpy as np

In [6]:
cur_path = Path(os.getcwd())
raw = cur_path / '..' / '..' / 'data' / '01_raw'

In [7]:
df = pd.read_csv(raw / 'sneaker.csv')
shoes = list(df.style_code)

In [8]:
def get_random_ua():
    random_ua = ''
    ua_file = raw / 'web-drivers' / 'user-agents.txt'
    try:
        with open(ua_file) as f:
            lines = f.readlines()
        if len(lines) > 0:
            prng = np.random.RandomState()
            index = prng.permutation(len(lines) - 1)
            idx = np.asarray(index, dtype=np.integer)[0]
            random_ua = lines[int(idx)]
            random_ua = random_ua[:len(random_ua)-2]
    except Exception as ex:
        print('Exception in random_ua')
        print(str(ex))
    finally:
        return random_ua

In [14]:
kixify = {}
long_delays = [i for i in range(10,21)]
delays = [i for i in range(3, 20)]

# for i in range(5):
for index, shoe in enumerate(shoes):
    if shoe in kixify.keys():
        continue
        
    url = 'https://www.goat.com/search?query=' + str(shoe)

    try:
        user_agent = get_random_ua()
        headers = {'user-agent': user_agent}
        source = BeautifulSoup(requests.get(url, headers=headers).text)
        time.sleep(np.random.choice(delays))
        prices = source.find_all('span', class_='product-price')
        all_prices = []
        for price in prices:
            p = str(price.text)
            p = p.replace(',', '')
            p = float(p[1:])
            all_prices.append(p)
        
        kixify[shoe] = {'med_price':np.median(all_prices), 'mean_price':np.mean(all_prices)}
        time.sleep(np.random.choice(long_delays))
    except:
        pass    

In [ ]:
with open(raw / 'kixify.json', 'w') as outfile:
    json.dump(kixify, outfile)